In [1]:
import ee
import geopandas as gpd
import json
import pandas as pd
import geemap


# https://developers.google.com/earth-engine/guides/service_account#use-a-service-account-with-a-private-key

service_account = 'gee-ine@ee-ricpizarrosine.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'privatekey.json')
ee.Initialize(credentials)

In [4]:
# I import this data from a .csv file, but to make a running exmample, here I create the DataFrame manually
data = [['Site-1', -42.44, 145.73],
        ['Site-2', -42.48, 146.36],
        ['Site-3', -42.14, 146.04]]
df = pd.DataFrame(data, columns = ['Name', 'Lat', 'Lon'])
 
df

,Name,Lat,Lon
0,Site-1,-42.44,145.73
1,Site-2,-42.48,146.36
2,Site-3,-42.14,146.04


In [ ]:
# Convert the dataframe to FeatureCollection
features = []
for index, row in df.iterrows():
    g = ee.Geometry.Point([row["Lon"], row["Lat"]])
    feature = ee.Feature(g, {"Name": ee.String((row["Name"]))})
    features.append(feature)

pts = ee.FeatureCollection(features)


In [13]:
# Create different buffer sizes around each point
def makeBuffer(size):
    def addBuffer(feature):
        return feature.buffer(size)
    return addBuffer
pts1000 = pts.map(makeBuffer(1000))
pts5000 = pts.map(makeBuffer(5000))
pts10000 = pts.map(makeBuffer(10000))



In [14]:
#### Extract the mean NDVI for each Buffer ####
startDate = '2001-02-01'
endDate = '2003-03-31'

modisNDVI = ee.ImageCollection('MODIS/MOD09GA_006_NDVI').select('NDVI').filterDate(startDate, endDate)



In [17]:
# Reducing across year-month
def calcNDVI(n):
    date = ee.Date(startDate).advance(n,'month')
    m = date.get("month")
    y = date.get("year")
    tempNDVI = (modisNDVI.filter(ee.Filter.calendarRange(y, y, 'year'))
                .filter(ee.Filter.calendarRange(m, m, 'month'))
                .mean()
                .set('year',y)
                .set('month',m))
    return tempNDVI            

# calculate the number of months to iterate
StartD = ee.Date(startDate)
EndD = ee.Date(endDate)
num_months = ((EndD.get("year").getInfo() - StartD.get("year").getInfo()) * 12 +
              (EndD.get("month").getInfo() - StartD.get("month").getInfo()))

# iterate
modis_YrMo = ee.List.sequence(0, num_months).map(calcNDVI)


In [19]:

### Calculate the mean NDVI inside each polygon(buffer)
def calcmean(fc):
    def regionmean(image):
      reduced = image.reduceRegions(
                  collection = fc,
                  reducer = ee.Reducer.mean(),
                  scale = 30)
      return reduced
    return regionmean

finalNDVI_1000 = ee.ImageCollection(modis_YrMo).map(calcmean(pts1000)).flatten()
df = geemap.ee_to_pandas(finalNDVI_1000)

print(modis_YrMo.getInfo()[1])

ModuleNotFoundError: No module named 'geemap'

In [37]:
import datetime
import ee

#Feature collection
shp_path = "ips_shapes/ips_shapes.shp"

shp_path = "ips_shapes/ips_shapes.shp"

# we read our shapefile in to a geopandas data frame using the geopandas.read_file method
# we'll make sure it's initiated in the EPSG 4326 CRS
shp_gdf = gpd.read_file(shp_path, crs='EPSG:4326')


In [38]:
# define a helper function to put the geodataframe in the right format for constructing an ee object
def shp_to_ee_fmt(geodf):
    data = json.loads(geodf.to_json())
    return data['features'][0]['geometry']['coordinates']

shp = ee.Geometry.MultiPolygon(shp_to_ee_fmt(shp_gdf))
#print(shp)

In [39]:
shp.geometry()

AttributeError: 'Geometry' object has no attribute 'geometry'

In [45]:
import datetime
import ee

#Feature collection
fc = ee.FeatureCollection("ft:1s57dkY_Sg_E_COTe3sy1tIR_U-5Gw-BQNwHh4Xel");
fc_filtered = fc.filter(ee.Filter.equals('NUM_DECS', 1))
#Image collection
Sentinel_collection = (ee.ImageCollection('COPERNICUS/S2')
    .filterBounds(fc_filtered)
    .filterDate(ee.Date('2017-01-01'),ee.Date('2017-08-01')))

def GetSeries(feature):
  def NDVIcalc(img):
    red = img.select('B4')
    nir = img.select('B8')
    ndvi = nir.subtract(red).divide(nir.add(red)).rename(['NDVI'])
    return (feature
            .set(ndvi.reduceRegion(ee.Reducer.mean(), feature.geometry(), 10))
            .set('date', img.date().format("YYYYMMdd")))

  series = Sentinel_collection.map(NDVIcalc)
# Get the time-series of values as two lists.
  list = series.reduceColumns(ee.Reducer.toList(2), ['date', 'NDVI']).get('list')
  return feature.set(ee.Dictionary(ee.List(list).flatten()))

result = fc_filtered.map(GetSeries)
print(result.getInfo())

EEException: Collection.loadTable: Collection asset 'ft:1s57dkY_Sg_E_COTe3sy1tIR_U-5Gw-BQNwHh4Xel' not found.

In [46]:
import ee

# https://developers.google.com/earth-engine/guides/service_account#use-a-service-account-with-a-private-key

service_account = 'gee-ine@ee-ricpizarrosine.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, 'privatekey.json')
ee.Initialize(credentials)

#Set the start date and end date
start_date = '2017-10-08'
end_date = '2017-10-31'

start_date = ee.Date(start_date)
end_date = ee.Date(end_date)

#Grab image collection
gridmet_db = 'IDAHO_EPSCOR/GRIDMET'
gridmet_raw= ee.ImageCollection(gridmet_db)

#Load California Boundary from TIGER collection
cal_bounds = ee.FeatureCollection('TIGER/2016/States').filter(ee.Filter.Or(ee.Filter.eq('NAME','California')))
california = cal_bounds.geometry().bounds()

#Create the table
def return_triplet(image):

    def set_properties(f):
        return f.set('imageId',image.id()).set('system:index',image.get('system:index')).set('system:time_start',image.get('system:time_start'))

    return image.reduceRegions(    
        collection=california,
        reducer=ee.Reducer.mean(),
        scale=2500).map(set_properties)

def set_properties(f):
    return f.set('imageId',image.id()).set('system:index',image.get('system:index')).set('system:time_start',image.get('system:time_start'))

export_triplets_mean = gridmet_raw.filterDate(start_date,end_date).map(return_triplet).flatten()
export_triplets_mean.aggregate_array('bi').getInfo()